In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup

In [2]:
ticker = 'AAPL'

def get_cashflow_statement(ticker):

    url = f'https://stockanalysis.com/stocks/{ticker}/financials/cash-flow-statement/'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html')

    cash_flow_table = soup.find('table', class_ = "w-full border-separate border-spacing-0 whitespace-nowrap")

    # Extract the column headers
    headers = []
    for header in cash_flow_table.find_all('th'):
        headers.append(header.text.strip())

    # Extract the table rows
    rows = []
    for row in cash_flow_table.find_all('tr')[1:]:  # Skipping the header row
        cells = row.find_all('td')
        row_data = [cell.text.strip() for cell in cells]
        rows.append(row_data)

    # Convert the data into a DataFrame and drop the last column that is paywalled
    cash_flow_statement = pd.DataFrame(rows, columns=headers)
    cash_flow_statement = cash_flow_statement.drop('2013 - 1992', axis=1)
    return cash_flow_statement

get_cashflow_statement(ticker)


,Year,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014
0,Net Income,"96,995","99,803","94,680","57,411","55,256","59,531","48,351","45,687","53,394","39,510"
1,Depreciation & Amortization,"11,519","11,104","11,284","11,056","12,547","10,903","10,157","10,505","11,257","7,946"
2,Share-Based Compensation,"10,833","9,038","7,906","6,829","6,068","5,340","4,840","4,210","3,586","2,863"
3,Other Operating Activities,"-8,804","2,206","-9,832","5,378","-4,480","1,660",877,"5,829","13,029","9,394"
4,Operating Cash Flow,"110,543","122,151","104,038","80,674","69,391","77,434","64,225","66,231","81,266","59,713"
5,Operating Cash Flow Growth,-9.50%,17.41%,28.96%,16.26%,-10.39%,20.57%,-3.03%,-18.50%,36.09%,11.27%
6,Capital Expenditures,"-10,959","-10,708","-11,085","-7,309","-10,495","-13,313","-12,451","-12,734","-11,488","-9,813"
7,Acquisitions,0,0,0,"-1,524",-624,-721,-329,-297,-343,"-3,765"
8,Change in Investments,"16,001","-9,560","-3,075","5,453","58,093","30,845","-33,542","-32,022","-44,417","-9,027"
9,Other Investing Activities,"-1,337","-2,086",-385,-909,"-1,078",-745,-124,-924,-26,26


In [4]:
def get_income_statement(ticker):
    url = f'https://stockanalysis.com/stocks/{ticker}/financials/'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html')

    income_statement = soup.find('table', class_ = 'w-full border-separate border-spacing-0 whitespace-nowrap')
    # Extract the column headers
    headers = []
    for header in income_statement.find_all('th'):
        headers.append(header.text.strip())

    # Extract the table rows
    rows = []
    for row in income_statement.find_all('tr')[1:]:  # Skipping the header row
        cells = row.find_all('td')
        row_data = [cell.text.strip() for cell in cells]
        rows.append(row_data)

    income_statement = pd.DataFrame(rows, columns=headers)
    income_statement = income_statement.drop('2013 - 1992', axis=1)
    return income_statement
    

In [5]:
def get_fcf(ticker):
    cash_flow_table = get_cashflow_statement(ticker)
    # Assuming 'Free Cash Flow' is in the first column, find the row index
    
    fcf_row = cash_flow_table[cash_flow_table.iloc[:, 0] == 'Free Cash Flow']
    return fcf_row

def mean_fcf(ticker):
    fcf_row = get_fcf(ticker)
    fcf_numeric = fcf_row.iloc[0, 1:]

    fcf_cleaned = []
    for value in fcf_numeric:
            # Remove any formatting and convert to float
            try:
                # Remove commas, dollar signs, and handle negative numbers in parentheses
                value_clean = value.replace(',', '').replace('$', '').replace('(', '-').replace(')', '')
                fcf_cleaned.append(float(value_clean))
            except ValueError:
                # If the value cannot be converted to float, ignore it
                continue
    

    return np.mean(fcf_cleaned) if fcf_cleaned else "No numeric data found"

In [6]:
def get_net_income(ticker):
    cash_flow_table = get_cashflow_statement(ticker)
    net_income = cash_flow_table[cash_flow_table.iloc[:, 0] == 'Net Income']
    return net_income

def avg_growth_rate_income(ticker):
    net_income = get_net_income(ticker)
    
    # Remove the 'Net Income' label and convert the data to numeric
    net_income = net_income.iloc[:, 1:].apply(lambda x: pd.to_numeric(x.str.replace(',', ''), errors='coerce'))
    
    #Ensure the data is in the correct order (if needed)
    net_income = net_income.iloc[:, ::-1]
    
    # Calculate yearly growth rates
    yearly_growth_rates = net_income.pct_change(axis=1) 

    # Calculate average growth rate
    average_growth_rate = yearly_growth_rates.mean(axis=1).iloc[0]
    return average_growth_rate 


def get_FCFtoNI_ratio(ticker):
    fcf = get_fcf(ticker)
    net_income = get_net_income(ticker)

    newdf = pd.concat([fcf, net_income], axis= 0)
    newdf.reset_index(drop=True, inplace= True)
  
    # Ensure the DataFrame columns are appropriate for calculations
    newdf.iloc[:, 1:] = newdf.iloc[:, 1:].map(lambda x: x.replace(',', '') if isinstance(x, str) else x)
    newdf.iloc[:, 1:] = newdf.iloc[:, 1:].map(lambda x: int(x) if isinstance(x, str) and x.isdigit() else x)


    fcf_net_income_ratio = newdf.iloc[0, 1:] / newdf.iloc[1, 1:]

    # # Add the ratio as a new row in the DataFrame
    fcf_net_income_ratio = pd.DataFrame(fcf_net_income_ratio).T
    fcf_net_income_ratio.insert(0, 'Year', 'FCF/Net Income Ratio')

    finaldf = pd.concat([newdf, fcf_net_income_ratio], axis= 0)
    finaldf.reset_index(drop= True, inplace=True)
    return finaldf


get_FCFtoNI_ratio(ticker)

,Year,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014
0,Free Cash Flow,99584,111443,92953,73365,58896,64121,51774,53497,69778,49900
1,Net Income,96995,99803,94680,57411,55256,59531,48351,45687,53394,39510
2,FCF/Net Income Ratio,1.026692,1.11663,0.98176,1.277891,1.065875,1.077103,1.070795,1.170946,1.306851,1.262971
